# Hands-On Pertemuan 3: Implementasi MapReduce Sederhana

## Tujuan:
- Memahami konsep MapReduce dalam distribusi data besar.
- Mengimplementasikan algoritma sederhana menggunakan MapReduce pada Hadoop.
- Melakukan analisis hasil MapReduce pada dataset.

### 1. Pengenalan MapReduce
MapReduce adalah model pemrograman yang digunakan untuk pemrosesan data besar secara paralel di beberapa node dalam kluster Hadoop.
- **Map**: Fase pertama di mana data dipecah menjadi unit-unit kecil (key-value pairs).
- **Reduce**: Fase kedua di mana hasil dari fase Map dikombinasikan untuk menghasilkan output yang lebih kecil.

- **Tugas 1**: Pelajari bagaimana MapReduce bekerja dengan dataset sederhana dan coba implementasikan konsep key-value pair.

### 2. Implementasi Sederhana: Word Count
Algoritma Word Count adalah salah satu contoh sederhana dari MapReduce. Dalam tugas ini, kita akan menghitung jumlah kata dalam dataset.

1. **Map Function**: Fungsi yang memecah teks menjadi kata-kata individual.
   ```python
   def map_function(text):
       for word in text.split():
           yield (word, 1)
   ```
2. **Reduce Function**: Fungsi yang menggabungkan hasil dari fase Map untuk menghitung frekuensi kata.
   ```python
   from collections import defaultdict

   def reduce_function(pairs):
       result = defaultdict(int)
       for word, count in pairs:
           result[word] += count
       return result
   ```
- **Tugas 2**: Implementasikan fungsi `map_function` dan `reduce_function` pada dataset teks sederhana, lalu hitung jumlah kata.

In [1]:
# 1. Definisi Map Function
def map_function(text):
    for word in text.split():
        yield (word, 1)

# 2. Definisi Reduce Function
from collections import defaultdict

def reduce_function(pairs):
    result = defaultdict(int)
    for word, count in pairs:
        result[word] += count
    return result

# Fungsi untuk membaca file teks
def read_file(file_path):
    with open(file_path, 'r') as file:
        text_data = file.readlines()
    return text_data

# Path ke file teks
file_path = 'tugasbigdata.txt'  # Sesuaikan dengan lokasi file Anda

# Membaca data dari file
text_data = read_file(file_path)

# Langkah Map: Memetakan setiap kalimat ke dalam pasangan (word, 1)
mapped_data = []
for text in text_data:
    mapped_data.extend(map_function(text))

# Langkah Reduce: Menghitung total kemunculan setiap kata
word_count = reduce_function(mapped_data)

print("Jumlah kata:", word_count)


Jumlah kata: defaultdict(<class 'int'>, {'Cristiano': 1, 'Ronaldo': 2, 'dan': 1, 'Lionel': 1, 'Messi': 2, 'adalah': 1, 'GOAT': 1, '(Greates': 1, 'of': 1, 'All': 1, 'Time).': 1, 'Namun': 1, 'ternyata': 1, 'setelah': 1, 'mendapat': 1, 'Pildun,': 1, 'nangis': 1, 'di': 1, 'bangku': 1, 'cadangan': 1})


### 3. Menjalankan Word Count di Hadoop
1. Siapkan file teks yang ingin dihitung jumlah katanya.
   ```bash
   hdfs dfs -put input.txt /user/student/input/
   ```
2. Jalankan perintah MapReduce pada file tersebut:
   ```bash
   hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-*.jar wordcount /user/student/input/ /user/student/output/
   ```
3. Tampilkan hasilnya:
   ```bash
   hdfs dfs -cat /user/student/output/part-r-00000
   ```
- **Tugas 3**: Upload file teks ke HDFS, jalankan perintah MapReduce Word Count, dan tampilkan hasilnya.

### 4. Menganalisis Dataset Besar
Untuk memperdalam pemahaman, kita akan menjalankan algoritma MapReduce pada dataset yang lebih besar.
- Pilih dataset besar yang dapat diunduh dari [Kaggle](https://www.kaggle.com/) atau sumber lain.
- Jalankan Word Count pada dataset besar tersebut dan analisis hasilnya.

- **Tugas 4**: Cari dataset besar, jalankan MapReduce untuk menghitung kata, dan buat laporan analisis hasil.

### 5. Tugas Tambahan: Custom MapReduce Algorithm
- Buat algoritma MapReduce lainnya, seperti menghitung rata-rata nilai, atau menghitung frekuensi kemunculan elemen tertentu di dalam dataset.

In [ ]:
from collections import defaultdict

# Fungsi untuk membaca file teks
def read_file(file_path):
    # Baca setiap baris dan pisahkan berdasarkan titik koma (CSV)
    with open(file_path, 'r') as file:
        text_data = [line.strip().split(';') for line in file.readlines()]
    return text_data

# Fungsi untuk memetakan negara menjadi pasangan (negara, 1)
def map_function(text):
    # Ambil kolom ketiga (CountryName) dan kelima (Year) jika tersedia
    if len(text) >= 6 and text[2].strip() and text[4].strip():  # Pastikan kolom CountryName dan Year tidak kosong
        country = text[2].strip()
        year = text[4].strip()
        return [(country, 1)]  # Kembalikan pasangan (CountryName, 1)
    return []

# Fungsi untuk menggabungkan jumlah kemunculan setiap negara
def reduce_function(mapped_data):
    country_count = defaultdict(int)
    for country, count in mapped_data:
        country_count[country] += count
    return dict(country_count)

# Path ke file teks
file_path = 'EconomyData.csv'  # Sesuaikan dengan lokasi file Anda

# Membaca data dari file
text_data = read_file(file_path)

# Langkah Map: Memetakan setiap negara ke dalam pasangan (negara, 1)
mapped_data = []
for row in text_data[1:]:  # Skip header baris pertama
    mapped_data.extend(map_function(row))

# Langkah Reduce: Menghitung jumlah kemunculan setiap negara
country_count = reduce_function(mapped_data)

# Menghitung total kemunculan semua negara
total_occurrences = sum(country_count.values())

# Menghitung rata-rata kemunculan per negara
average_occurrences = total_occurrences / len(country_count)

# Menampilkan hasil setiap negara dan jumlah kemunculannya serta rata-rata
print("Country | Jumlah Kemunculan | Rata-rata Kemunculan")
print("-" * 50)
for country, count in country_count.items():
    print(f"{country}: {count} | {(count / total_occurrences):.2%}")

# Menampilkan total dan rata-rata kemunculan
print(f"\nTotal Kemunculan Semua Negara: {total_occurrences}")
